## Build the structured data files from the raw data:

In [1]:
from fpl_predictor.build_data import DataBuilder

# Run this code to update the structured data:
db = DataBuilder()

# Alternatively run this to access the object without building files:
db = DataBuilder(build_player_files=False, build_master_csv=False)

# Example function:
db.load_year_players(2019).head()

Making master player files ... ✓
Making master player gameweek CSV file ... ✓


,name,code,element_type,first_name,id,second_name,squad_number,team,team_code,web_name
0,Shkodran Mustafi,69140,2,Shkodran,1,Mustafi,None,1,3,Mustafi
1,Héctor Bellerín,98745,2,Héctor,2,Bellerín,None,1,3,Bellerín
2,Sead Kolasinac,111457,2,Sead,3,Kolasinac,None,1,3,Kolasinac
3,Ainsley Maitland-Niles,154043,2,Ainsley,4,Maitland-Niles,None,1,3,Maitland-Niles
4,Sokratis Papastathopoulos,39476,2,Sokratis,5,Papastathopoulos,None,1,3,Sokratis


## Get information about players:

In [2]:
from fpl_predictor.player_information import PlayerInformation
pi = PlayerInformation()
pi.search_players("trent", "al")

{169187: 'Trent Alexander-Arnold'}

In [3]:
# Get more details from a specific gameweek:
code = 169187
pi.get_player(code, year=2019, week=24)

{'code': 169187,
 'name': 'Trent Alexander-Arnold',
 'position': 'DEF',
 'value': 76,
 'team': 'Liverpool'}

## Build a Squad:

In [1]:

import json
import os
import pandas as pd

from fpl_predictor.api import ApiData
from fpl_predictor.functions import index_players
from fpl_predictor.dtypes import csvtypes
from nav import DIR_STRUCTURED_DATA, DIR_STATIC


class PlayerInformation:
    """Class for loading player details such as value, position, and team for a
    given year-week."""

    def __init__(self):
        self._api_data = ApiData()
        fp = os.path.join(DIR_STRUCTURED_DATA, "master.csv")
        df = pd.read_csv(fp, encoding="utf-8", dtype=csvtypes)
        self._master = df

    def _master_year_week(self, year, week):
        """Get a subset of the master DF for the given year-week."""
        assert 0 < week < 39, f"Week must be int from 1 to 38"
        df = self._master.copy()
        df = df.loc[(df["year"] == year) & (df["GW"] == week)]
        return df.reset_index(drop=True)

    @property
    def _team_codes(self):
        """Maps team codes to full team names."""
        fp = os.path.join(DIR_STATIC, "team_codes.json")
        with open(fp, "r") as f:
            return {int(k): v for k, v in json.load(f).items()}

    def player_teams(self, year: int, live=False):
        """Dictionary of player codes to team in year."""
        if live:
            df = self._api_data.elements[["code", "team_code"]].copy()
            df["team"] = df["team_code"].map(self._team_codes)
            return dict(zip(df["code"], df["team"]))
        else:
            fp = os.path.join(DIR_STRUCTURED_DATA, "year_player_team.json")
            with open(fp, "r") as f:
                d = json.load(f)[str(year)]
            d = {int(k): self._team_codes[int(v)] for k, v in d.items()}
            return d

    def player_names(self, live=False):
        """Dictionary of player codes to player names."""
        if live:
            df = self._api_data.elements[["code", "first_name", "second_name"]].copy()
            df["name"] = df["first_name"].str.cat(df["second_name"], sep=" ")
            return dict(zip(df["code"], df["name"]))
        else:
            fp = os.path.join(DIR_STRUCTURED_DATA, "player_names.json")
            with open(fp, "r") as f:
                return {int(k): v for k, v in json.load(f).items()}

    def search_players(self, *strings: str, live: bool = False):
        """Search for player names matching the supplied strings to get their
        unqiue codes."""
        strings = [s.lower() for s in strings]
        names = self.player_names(live=live)
        return {k: v for k, v in names.items() if all([s in str.lower(v) for s in strings])}

    def player_values(self, year: int = None, week: int = None, live=False):
        """Dictionary of player codes to value in year-week."""
        if live:
            df = self._api_data.elements
            return dict(zip(df["code"], df["now_cost"]))
        else:
            df = self._master_year_week(year, week)
            return dict(zip(df["code"], df["value"]))

    def player_positions(self, year: int = None, live=False):
        """Dictionary of player codes to position in year."""
        if live:
            df = self._api_data.elements[["code", "element_type"]].copy()
            df["position"] = df["element_type"].map({1: "GK", 2: "DEF", 3: "MID", 4: "FWD"})
            return dict(zip(df["code"], df["position"]))
        else:
            fp = os.path.join(DIR_STRUCTURED_DATA, "year_player_position.json")
            with open(fp, "r") as f:
                d = json.load(f)[str(year)]
            return {int(k): v for k, v in d.items()}

    def add_player_info_to_df(self, df, year: int = None, week: int = None,
                              live=False, code_col="code"):
        """Adds all player details to a Pandas DataFrame, which must have a
        `code_col` column with the unique player id."""
        df = df.copy()
        df["position"] = df[code_col].map(self.player_positions(year=year, live=live))
        df["value"] = df[code_col].map(self.player_values(year=year, week=week, live=live))
        df["team"] = df[code_col].map(self.player_teams(year=year, live=live))
        df["name"] = df[code_col].map(self.player_names(live=live))
        return df

    def points_scored(self, year: int = None, week: int = None):
        """Dictionary of player codes to total points they earnt in a given
        week. Can only query historical data."""
        df = self._master_year_week(year, week)
        return dict(zip(df["code"], df["total_points"]))

    def minutes_played(self, year: int = None, week: int = None):
        """Dictionary of player codes to minutes played in the given year-week.
        """
        df = index_players(self._master, column="minutes")
        return df.loc[(year, week), :].to_dict()

    def get_player(self, code: int, year: int = None, week: int = None,
                   live=False):
        """Get a dict of player information from their code."""
        player = dict(code=code, name=self.player_names()[code],
                      position=self.player_positions(year=year, live=live)[code],
                      value=self.player_values(year=year, week=week, live=live)[code],
                      team=self.player_teams(year=year, live=live)[code])
        return player


In [2]:
pi = PlayerInformation()

In [6]:
# Use this object to build a squad:
from fpl_predictor.squad_builder import SquadBuilder

# Scoring metric is the column from the data that you want to rank players by:
sqb = SquadBuilder(scoring_metric="ict_index", n=23, cross_seasons=False)

/Users/tp/virtualenvs/fpl-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  """
/Users/tp/virtualenvs/fpl-predictor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# Use a filling strategy and optimizing strategy to build a squad from scratch:
from fpl_predictor.strategies.sample import build
from fpl_predictor.strategies.optimisers.player_permutations import optimiser
from fpl_predictor.strategies.builders.builders import top_fill, middle_fill

sqb.build_squad(top_fill, 2019, 23, live=False)
sqb.optimise_squad(optimise_function=optimiser, n_iterations=100, year=2019, week=23, live=True)

IndexError: single positional indexer is out-of-bounds

In [6]:
sqb.squad.selected

,code,name,position,value,score,score_per_value,team
0,103955,Raheem Sterling,MID,120.0,12.100000,0.100833,Man City
1,61366,Kevin De Bruyne,MID,102.0,11.063636,0.108467,Man City
2,169187,Trent Alexander-Arnold,DEF,72.0,9.500000,0.131944,Liverpool
3,57127,Teemu Pukki,FWD,66.0,8.300000,0.125758,Norwich
4,184341,Mason Mount,MID,67.0,8.090909,0.120760,Chelsea
5,195546,Emiliano Buendía,MID,60.0,7.945455,0.132424,Norwich
6,103123,Sébastien Haller,FWD,71.0,7.790909,0.109731,West Ham
7,114283,Jack Grealish,MID,59.0,7.500000,0.127119,Aston Villa
8,101188,Lucas Digne,DEF,58.0,6.936364,0.119592,Everton
9,115382,Neal Maupay,FWD,59.0,6.890909,0.116795,Brighton


In [40]:
# Load a squad from CSV:
sqb.squad.load_squad("gw24")

Squad of 15 players loaded from /Users/tp/projects/fpl-predictor/data/squads/gw24.csv


In [41]:
# See the first team chosen by the scoring_metric:
sqb.squad.selected

,code,name,position,value,score,score_per_value,team
0,102057,Raúl Jiménez,FWD,76,13.5,0.177632,Wolves
1,118748,Mohamed Salah,MID,126,12.1,0.096032,Liverpool
2,108823,Bamidele Alli,MID,85,11.8,0.138824,Spurs
3,114283,Jack Grealish,MID,67,10.1,0.150746,Aston Villa
4,193111,Todd Cantwell,MID,48,9.2,0.191667,Norwich
5,169187,Trent Alexander-Arnold,DEF,72,9.0,0.125000,Liverpool
6,171129,Diego Rico,DEF,42,5.8,0.138095,Bournemouth
7,101668,Jamie Vardy,FWD,99,3.3,0.033333,Leicester
8,172850,Benjamin Chilwell,DEF,56,3.2,0.057143,Leicester
9,98747,Nick Pope,GK,46,3.1,0.067391,Burnley


In [37]:
sqb.add_player(code=178304, year=2019, week=24, live=True)

Player added to squad: Lys Mousset


In [36]:
pi.search_players("mous")

{178304: 'Lys Mousset',
 39104: 'Mousa Dembélé',
 45268: 'Moussa Sissoko',
 431131: 'Moussa Djenepo'}

In [18]:
pi.get_player(58786, 2019, 24)

{'code': 58786,
 'name': 'Martin Kelly',
 'position': 'DEF',
 'value': 44,
 'team': 'Crystal Palace'}

In [9]:
sqb.squad.substitutes

,code,name,position,value,score,score_per_value,team
0,184341,Mason Mount,MID,67,3.1,0.046269,Chelsea
1,153723,John Lundstram,DEF,42,2.5,0.059524,Sheffield Utd
2,126184,Nathan Aké,DEF,50,1.0,0.020000,Bournemouth
3,38533,Rui Pedro dos Santos Patrício,GK,51,1.0,0.019608,Wolves


In [10]:
sqb.squad.first_team

,code,name,position,score,score_per_value,team,value
0,98747,Nick Pope,GK,3.1,0.067391,Burnley,46.0
1,176413,Christian Pulisic,MID,18.6,0.254795,Chelsea,73.0
2,103955,Raheem Sterling,MID,17.3,0.140650,Man City,123.0
3,85971,Heung-Min Son,MID,10.2,0.104082,Spurs,98.0
4,193111,Todd Cantwell,MID,9.2,0.191667,Norwich,48.0
5,57127,Teemu Pukki,FWD,9.0,0.128571,Norwich,70.0
6,173879,Tammy Abraham,FWD,8.6,0.111688,Chelsea,77.0
7,80146,Jordan Ayew,FWD,6.0,0.117647,Crystal Palace,51.0
8,171129,Diego Rico,DEF,5.8,0.138095,Bournemouth,42.0
9,39487,Erik Pieters,DEF,3.5,0.074468,Burnley,47.0


## Captain & Vice-Captain Picks

In [11]:
sqb.squad.captain

{176413: 'Christian Pulisic'}

In [12]:
sqb.squad.vice_captain

{103955: 'Raheem Sterling'}

## Evaluate potential transfers:

In [13]:
# Evaluate potential transfers from the currently selected squad:
from fpl_predictor.strategies.sample import evaluate_transfers, make_tranfers

# Change the criteria for ranking players:
sqb.cross_seasons = True
sqb.set_n(5)
sqb.set_scoring_metric("ict_index")

# Get suggested transfers:
transfers = evaluate_transfers(sqb, 2019, 14, live=True)
transfers

,available,code,minutes_percent,name,out_code,out_name,out_score,out_score_per_value,out_team,out_value,position,score,score_per_value,team,value,score_gain,score_per_val_gain,val_gain,squad_unavailable,first_team_unavailable
0,1.0,101668.0,1.00,Jamie Vardy,173879.0,Tammy Abraham,8.6,0.111688,Chelsea,77.0,FWD,12.86,0.131224,Leicester,98.0,4.26,0.019536,-21.0,True,True
1,1.0,172780.0,0.96,James Maddison,193111.0,Todd Cantwell,9.2,0.191667,Norwich,48.0,MID,11.32,0.148947,Leicester,76.0,2.12,-0.042719,-28.0,True,True
2,1.0,169187.0,1.00,Trent Alexander-Arnold,126184.0,Nathan Aké,1.0,0.020000,Bournemouth,50.0,DEF,9.44,0.131111,Liverpool,72.0,8.44,0.111111,-22.0,False,False
3,1.0,172780.0,0.96,James Maddison,184341.0,Mason Mount,3.1,0.046269,Chelsea,67.0,MID,11.32,0.148947,Leicester,76.0,8.22,0.102679,-9.0,False,False
4,1.0,169187.0,1.00,Trent Alexander-Arnold,153723.0,John Lundstram,2.5,0.059524,Sheffield Utd,42.0,DEF,9.44,0.131111,Liverpool,72.0,6.94,0.071587,-30.0,False,False
5,1.0,101668.0,1.00,Jamie Vardy,80146.0,Jordan Ayew,6.0,0.117647,Crystal Palace,51.0,FWD,12.86,0.131224,Leicester,98.0,6.86,0.013577,-47.0,False,False
6,1.0,169187.0,1.00,Trent Alexander-Arnold,172850.0,Benjamin Chilwell,3.2,0.057143,Leicester,56.0,DEF,9.44,0.131111,Liverpool,72.0,6.24,0.073968,-16.0,False,False
7,1.0,169187.0,1.00,Trent Alexander-Arnold,39487.0,Erik Pieters,3.5,0.074468,Burnley,47.0,DEF,9.44,0.131111,Liverpool,72.0,5.94,0.056643,-25.0,False,False
8,1.0,101668.0,1.00,Jamie Vardy,57127.0,Teemu Pukki,9.0,0.128571,Norwich,70.0,FWD,12.86,0.131224,Leicester,98.0,3.86,0.002653,-28.0,False,False
9,1.0,169187.0,1.00,Trent Alexander-Arnold,171129.0,Diego Rico,5.8,0.138095,Bournemouth,42.0,DEF,9.44,0.131111,Liverpool,72.0,3.64,-0.006984,-30.0,False,False


## Make Transfers:

In [14]:
sqb.squad.remove_player(126184)

Removed player from squad (14 remain): Nathan Aké


In [15]:
sqb.add_player(code=169187.0, year=2019, week=13, live=False)

Player added to squad: Trent Alexander-Arnold


In [39]:
# Save a squad to CSV:
sqb.squad.save_squad("gw24")

Squad of 15 players saved at: /Users/tp/projects/fpl-predictor/data/squads/gw24.csv
